In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn import impute
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('data/train.csv')

Add cabin later

In [3]:
train_columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
num_attribs = ['Pclass','Age', 'SibSp', 'Parch', 'Fare',]
cat_attribs = ['Sex', 'Embarked']

In [4]:
df['Embarked'].fillna('N', inplace=True)

In [5]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df, df['Pclass']):
    df_train = df.iloc[train_index]
    df_test= df.iloc[test_index]

# Work on this later

In [6]:
[a[1::] for a in aa]

NameError: name 'aa' is not defined

In [ ]:
aa.unique()

# Main stuff

In [7]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [8]:
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', impute.SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('cat_encoder', OneHotEncoder())
])

full_pipeline = FeatureUnion(transformer_list =[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

In [9]:
X_train = full_pipeline.fit_transform(df_train).toarray()
y_train = df_train['Survived'].values

X_test = full_pipeline.transform(df_test).toarray()
y_test = df_test['Survived'].values

# Machine Learning Stuff

## SVM

In [10]:
%%capture
param_grid={'C': np.arange(1, 45, 3), 'gamma': np.arange(0.001, 0.2, 0.01), 'kernel': ('linear', 'rbf')}
SVC_srch = GridSearchCV(estimator=SVC(probability = True), param_grid=param_grid,
                 verbose=2, cv=4)
SVC_srch.fit(X_train, y_train)

KeyboardInterrupt: 

In [11]:
results = SVC_srch.cv_results_

Cs = np.array(results['param_C'], dtype=float)
gammas = np.array(results['param_gamma'], dtype=float)
scores = np.array(results['mean_test_score'], dtype=float)

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

In [ ]:
plt.scatter(Cs, gammas, c=scores, cmap="jet", lw=0, vmin=0.8)
plt.xlabel('C')
plt.ylabel(r'$\gamma$')
plt.colorbar(label='Score')
plt.title('Scores')
plt.show()

In [ ]:
SVC_clf = SVC_srch.best_estimator_

## Random Forest

In [ ]:
%%capture
criterion = {1: 'gini', 2:'entropy'}
param_distributions = {"max_depth": np.arange(4, 20, 2),
                       "n_estimators": np.arange(10, 40, 4),
                       "criterion": ['gini', 'entropy']}

rnd_srch = GridSearchCV(RandomForestClassifier(), param_distributions,
                        verbose=2, cv=5)
rnd_srch.fit(X_train, y_train)

In [ ]:
rnd_clf = rnd_srch.best_estimator_

## Voting

In [ ]:
voting_clf = VotingClassifier(
    estimators=[('rf', rnd_clf), ('svc', SVC_clf)],
    voting='soft')
voting_clf.fit(X_train, y_train)

## Get all predictions

In [ ]:
# Print all scores
for clf in (rnd_clf, SVC_clf, voting_clf):
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

In [ ]:
rnd_clf.feature_importances_